In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
import json, os, progressbar, re, time

from Bio import SeqIO
from Bio import AlignIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from matplotlib_venn import venn3, venn3_circles
from matplotlib_venn import venn2, venn2_circles

from tqdm import tqdm 
from tqdm import trange
from plotnine import *

#### Loading input files

In [3]:
#### Species FASTA files
HG_fasta = '../../data/ortholog_dataset/uni_HG_orthologs.faa'
MM_fasta = '../../data/ortholog_dataset/uni_MM_orthologs.faa'
matcher_table = pd.read_csv('../../data/alignment_quality/HGMM_matcher_scores.csv')

In [4]:
#### Ortholog dataset
in_ortho_db = pd.read_csv('../../data/ortholog_dataset/HG_MM_Orthologs_Length.csv',sep='\t')
HG_ortho = in_ortho_db[['clusterNumber', 'proteinID_x']]
MM_ortho = in_ortho_db[['clusterNumber', 'proteinID_y']]

MM_IDs = [ seqRecord.id for seqRecord in SeqIO.parse(MM_fasta, format='fasta')]
HG_IDs = [ seqRecord.id for seqRecord in SeqIO.parse(HG_fasta, format='fasta')]

In [17]:
#### Whole-protein sequence aggregation propensity scores
all_agg_scores = pd.read_csv('../../data/aggregation_propensity/HGMM_agg_scores.csv', sep=',') 
all_agg_scores['delta_aggregation'] = all_agg_scores['Aggregation_x'] - all_agg_scores['Aggregation_y']
all_agg_scores['delta_agg_z-scores'] = stats.zscore(all_agg_scores['delta_aggregation'])

#### PFAM database
- Need to be downloaded with `pfam_download.py`

In [9]:
with open('../../data/domain_annotations/MM_results_pfam.json') as json_file:
    MM_data = json.load(json_file)

In [10]:
with open('../../data/domain_annotations//HG_results_pfam.json') as json_file:
    HG_data = json.load(json_file)

## 1. Collect number of domains for all proteins

In [11]:
def check_annotations(ID, fastaFile):
    for seqRecord in SeqIO.parse(fastaFile, format='fasta'):
        if ID in seqRecord.id :
            if '_MOUSE' in seqRecord.description:
                description = re.findall(r'_MOUSE (.*) OS', seqRecord.description)[0]
            if '_HETGA' in seqRecord.description:
                description = re.findall(r'_HETGA (.*) OS', seqRecord.description)[0]
            return description


def return_pfam_entry(proteinID, data):
     for i in range(len(data)):
        if data[i]['metadata']['accession'] == proteinID :
            return data[i]


def get_domain_nb(proteinID, data):
    json_result = return_pfam_entry(proteinID, data)
    cpt = 0
    for pfam_entry in json_result['entry_subset']:
        cpt += len(pfam_entry['entry_protein_locations'])
    return cpt


def domain_stat(IDs, data, model):
    tmp = []
    bar = progressbar.ProgressBar()
    for ID in bar(IDs):
        try:
            tmp.append((ID, get_domain_nb(ID, data)))
        except:
            tmp.append((ID, 0))
            
    if 'HG' in model:
        protein_type = 'proteinID_x'
    if 'MM' in model:
        protein_type = 'proteinID_y'
        
    stat_dom_table = pd.DataFrame(tmp, columns=[protein_type, 'nb_domains'])
    return stat_dom_table

In [12]:
MM_nb_dom = domain_stat(MM_IDs, MM_data, 'MM')

100% (13806 of 13806) |##################| Elapsed Time: 0:00:28 Time:  0:00:28


In [13]:
HG_nb_dom = domain_stat(HG_IDs, HG_data, 'HG')

100% (13806 of 13806) |##################| Elapsed Time: 0:01:02 Time:  0:01:02


### Restrict to ortholog dataset

In [18]:
HG_ortho_dom = HG_nb_dom[HG_nb_dom['proteinID_x'].isin(all_agg_scores['proteinID_x'])]
MM_ortho_dom = MM_nb_dom[MM_nb_dom['proteinID_y'].isin(all_agg_scores['proteinID_y'])]

In [19]:
ortho_pairs = all_agg_scores[['proteinID_x', 'proteinID_y']]
HG_ortho_dom = HG_ortho_dom.merge(ortho_pairs, on='proteinID_x')
MM_ortho_dom = MM_ortho_dom.merge(ortho_pairs, on='proteinID_y')
ALL_ortho_dom = HG_ortho_dom.merge(MM_ortho_dom, on=['proteinID_x', 'proteinID_y'])

In [21]:
ALL_ortho_dom = ALL_ortho_dom.sort_values('proteinID_x')
all_agg_scores['Annotations'] = (ALL_ortho_dom['nb_domains_x'] != 0) & (ALL_ortho_dom['nb_domains_y'] != 0) 

In [22]:
len(all_agg_scores[all_agg_scores['Annotations'] == True ])

6956

## 2. Build domain definition table 

In [23]:
def build_domain_def_table(ortho_dom, data, model):
    if 'HG' in model:
        protein_type = 'proteinID_x'
        header = [protein_type, 'domain_id', 'dom_start_x', 'dom_end_x']
    elif 'MM' in model:
        protein_type = 'proteinID_y'
        header = [protein_type, 'domain_id', 'dom_start_y', 'dom_end_y']
        
    tmp = []
    bar = progressbar.ProgressBar()
    for ID in bar(ortho_dom[ortho_dom['nb_domains'] != 0][protein_type]):
        json_result = return_pfam_entry(ID, data)
        for entry in json_result['entry_subset']:
            for positions in entry['entry_protein_locations']:
                model = positions['model']
                start = positions['fragments'][0]['start']
                end = positions['fragments'][0]['end']
                tmp.append((ID, model, int(start)-1, int(end)-1)) ### -1 as we are collecting an alignment position starting from 1
    
    dom_def = pd.DataFrame(tmp, columns=header)
    return dom_def
    

#### Statistics

In [24]:
MM_dom_def = build_domain_def_table(MM_ortho_dom, MM_data, 'MM')  
HG_dom_def = build_domain_def_table(HG_ortho_dom, HG_data, 'HG')  

100% (8519 of 8519) |####################| Elapsed Time: 0:00:19 Time:  0:00:19
100% (7352 of 7352) |####################| Elapsed Time: 0:00:30 Time:  0:00:30


In [25]:
print(f'{len(MM_dom_def)} annotated domains for {len(np.unique(MM_dom_def["proteinID_y"]))} mouse proteins')

19725 annotated domains for 8519 mouse proteins


In [26]:
print(f'Missing annotations for {len(MM_ortho_dom[MM_ortho_dom["nb_domains"] == 0])} proteins')

Missing annotations for 1003 proteins


In [27]:
print(f'{len(HG_dom_def)} annotated domains for {len(np.unique(HG_dom_def["proteinID_x"]))} naked-mole rat proteins')

17162 annotated domains for 7352 naked-mole rat proteins


In [28]:
print(f'Missing annotations for {len(HG_ortho_dom[HG_ortho_dom["nb_domains"] == 0])} proteins')

Missing annotations for 2170 proteins


## 3. Collect Tango scores for all domains in ortholog dataset

In [29]:
def collect_dom_scores(dom_def, model, ID):
    tmp = []
        
    if 'HG' in model:
        protein_type = 'proteinID_x'
        tango_output = '/media/savvy/DATA3/savvy/project_2018/WT_TANGO/HG'
        start_type = 'dom_start_x'
        end_type = 'dom_end_x'
    if 'MM' in model:
        protein_type = 'proteinID_y'
        tango_output = '/media/savvy/DATA3/savvy/project_2018/WT_TANGO/MM'
        start_type = 'dom_start_y'
        end_type = 'dom_end_y'
    
    agg_table = pd.read_csv(os.path.join(tango_output,f'{ID}.txt'), sep='\t')
    
    dom_table = dom_def[dom_def[protein_type] == ID].reset_index()        
    for idx in dom_table.index :
        start = int(dom_table[start_type][idx])
        end = int(dom_table[end_type][idx])
        agg_score = sum(agg_table['Aggregation'][start:end+1]) / len(agg_table['Aggregation'][start:end+1])
        tmp.append([ID, dom_table['domain_id'][idx], start, end, agg_score])
        
    return tmp

In [30]:
MM_DF = pd.DataFrame()
bar = progressbar.ProgressBar()
for ID in bar(MM_ortho_dom['proteinID_y']):
    DF_A = pd.DataFrame(collect_dom_scores(MM_dom_def, 'MM', ID), columns=['proteinID_y', 'domain_id', 'dom_start_y', 'dom_end_y', 'dom_agg_score_y'])
    MM_DF = MM_DF.append(DF_A)

100% (9522 of 9522) |####################| Elapsed Time: 0:02:09 Time:  0:02:09


In [31]:
#### Number of proteins with annotated domains in mouse 
len(np.unique(MM_DF['proteinID_y']))

8519

In [32]:
HG_DF = pd.DataFrame()
bar = progressbar.ProgressBar()
for ID in bar(HG_ortho_dom['proteinID_x']):
    DF_B = pd.DataFrame(collect_dom_scores(HG_dom_def, 'HG', ID), columns=['proteinID_x', 'domain_id', 'dom_start_x', 'dom_end_x', 'dom_agg_score_x'])
    HG_DF = HG_DF.append(DF_B)

100% (9522 of 9522) |####################| Elapsed Time: 0:02:07 Time:  0:02:07


In [33]:
#### Number of proteins with annotated domains in naked mole-rat 
len(np.unique(HG_DF['proteinID_x']))

7352

In [34]:
def collect_raw_pos(seqs, dom_table, idx):
    domain_id = dom_table['domain_id'][idx]
    start = dom_table['dom_start_y'][idx]
    end = dom_table['dom_end_y'][idx]

    dom_record = SeqRecord(
        Seq(str(seqs[0].seq)[start:end+1]), 
        id=seqs[0].id, 
        name=domain_id, 
        description=f'{seqs[0].id}_{domain_id}_{start}_{end}')
    
    with open("./tmp/dom.fasta", "w") as output_handle:
        SeqIO.write(dom_record, output_handle, format='fasta')
    
    query = "./tmp/dom.fasta"
    subject = "./tmp/subject.fasta"
    !blastp -query {query} -subject {subject} -evalue 10E-3 -outfmt 7 -out ./tmp/domain_mapping.csv
    try: 
        blast_res = pd.read_csv('./tmp/domain_mapping.csv', names=['query acc.ver', 'subject acc.ver', '% identity', 'alignment length', 'mismatches', 'gap opens', 'q. start', 'q. end', 's. start', 's. end', 'evalue', 'bit score'], comment='#', sep='\t')
        dom_start = blast_res['q. start'][0]
        dom_end = blast_res['q. end'][0]
        sub_start = blast_res['s. start'][0]
        sub_end = blast_res['s. end'][0]
        dom_patt = dom_record.seq[dom_start-1:dom_end+1]
        q_start = seqs[0].seq.find(dom_patt)
        q_end = q_start + len(dom_patt)-1
        return seqs[0].id, seqs[1].id, domain_id, int(q_start), int(q_end), int(sub_start), int(sub_end)
    
    except:
        print(f'No mapping for {seqs[0].id} - {domain_id} {start} {end}')
        return None
    

def get_sequences(y):
    tmp = []
    
    x = all_agg_scores[all_agg_scores['proteinID_y'] == y]['proteinID_x'].values[0]
    for seqRecord in SeqIO.parse(MM_fasta, format='fasta'):
        if y in seqRecord.id : 
            tmp.append(seqRecord)
    for seqRecord in SeqIO.parse(HG_fasta, format='fasta'):
        if x in seqRecord.id :
            tmp.append(seqRecord)
    return tmp

def collect_dom_pos(ID):
    ## Collect MM and HG sequences
    seqs = get_sequences(ID)
    with open("./tmp/query.fasta", "w") as output_handle:
        SeqIO.write(seqs[0], output_handle, format='fasta')
    with open("./tmp/subject.fasta", "w") as output_handle:
        SeqIO.write(seqs[1], output_handle, format='fasta')
    ## Collect domain table for x protein
    dom_table = MM_dom_def[MM_dom_def['proteinID_y'] == ID].reset_index()

    tmp = []
    for idx in dom_table.index:
        tmp.append(collect_raw_pos(seqs, dom_table, idx))
    return tmp

def get_dom_agg_score(model, dom_table):
    if 'HG' in model :
        tango_output = '/media/savvy/DATA3/savvy/project_2018/WT_TANGO/HG/'
        protein_type = 'proteinID_x'
        start_type = 'dom_start_x'
        end_type = 'dom_end_x'
    if 'MM' in model : 
        tango_output = '/media/savvy/DATA3/savvy/project_2018/WT_TANGO/MM/'
        protein_type = 'proteinID_y'
        start_type = 'dom_start_y'
        end_type = 'dom_end_y'
    
    try:
        ID = dom_table[protein_type].values[0]
    
        agg_table = pd.read_csv(os.path.join(tango_output,f'{ID}.txt'), sep='\t')
        tmp = []
        for idx in dom_table.index :
            start = dom_table[start_type][idx]
            end = dom_table[end_type][idx]
            agg_score = sum(agg_table['Aggregation'][int(start):int(end)+1]) / len(agg_table['Aggregation'][int(start):int(end)+1])
            tmp.append([ID, dom_table['domain_id'][idx], start, end, agg_score])
        return tmp
    
    except:
        return []

def domain_mapping(ID):
    tmp = collect_dom_pos(ID)
    if None in tmp :
        tmp.remove(None)
    try:
        if len(tmp) != 0 :
            dom_table = pd.DataFrame(tmp, columns=['proteinID_y', 'proteinID_x', 'domain_id', 'dom_start_y', 'dom_end_y', 'dom_start_x', 'dom_end_x'])
            dom_table = dom_table.dropna()

            MM_list = get_dom_agg_score('MM', dom_table)
            HG_list = get_dom_agg_score('HG', dom_table)

            tmp = []

            for i in range(len(MM_list)):
                proteinID_y = MM_list[i][0]
                domain_id = MM_list[i][1]
                dom_start_y = int(MM_list[i][2])
                dom_end_y = int(MM_list[i][3])
                dom_agg_score_y = MM_list[i][4]
                dom_length_y = dom_end_y - dom_start_y

                proteinID_x = HG_list[i][0]
                dom_start_x = int(HG_list[i][2])
                dom_end_x = int(HG_list[i][3])
                dom_agg_score_x = HG_list[i][4]
                dom_length_x = dom_end_x - dom_start_x

                tmp.append([proteinID_y, domain_id, dom_start_y, dom_end_y, dom_agg_score_y, dom_length_y, proteinID_x, dom_start_x, dom_end_x, dom_agg_score_x, dom_length_x])
            return ID, pd.DataFrame(tmp, columns=['proteinID_y', 'domain_id', 'dom_start_y', 'dom_end_y', 'dom_agg_score_y', 'dom_length_y', 'proteinID_x', 'dom_start_x', 'dom_end_x', 'dom_agg_score_x', 'dom_length_x'])
        else:
            print(f'No annotations mapping possible for {ID}')
            return ID, []
    except:
        print(f'No annotations mapping possible for {ID}')
        return ID, []

In [ ]:
troubleshoot = []
dom_agg_table = pd.DataFrame()
bar = progressbar.ProgressBar()
for idx in bar(ortho_pairs.index) :
    proteinID_y = ortho_pairs['proteinID_y'][idx]
    ID, map_success = domain_mapping(proteinID_y)
    if len(map_success) != 0 :
        dom_agg_table = dom_agg_table.append(map_success)
    else: 
        troubleshoot.append(ID)

In [ ]:
dom_agg_table.to_csv('../../data/aggregation_propensity/HGMM_dom_agg_scores.csv', sep='\t', index=False)